In [1]:
from data import preparation
import data.hierarchy as hie
from sklearn.model_selection import train_test_split

In [2]:
data_name = "wiki_small"

In [3]:
with open('data/%s/hierarchy.txt' % data_name, 'w') as f1:
    with open('data/%s/%s.ht' % (data_name,data_name)) as f:
        for l in f:
            split = l.strip().split(':')
            p = split[0]
            c = split[1].split(',')
            for cc in c:
                f1.write('%s %s\n'% (p,cc))

In [4]:
hierarchy, parent_of, all_name, name_to_index, level = hie.reindex_hierarchy(
                '%s/hierarchy.txt' % data_name)
hie.save_hierarchy("%s/hierarchy.pickle" % data_name, hierarchy,
                   parent_of, all_name, name_to_index, level)

In [5]:
for i in range(len(level) - 1 ):
    print(level[i+1] - level[i])

18
281
1055
1884
1800
968
367
49
1


In [6]:
# for mode in ['train', 'test']:
#     for i in range(1,6):
#         datas, labels = preparation.import_data('%s/folds/%s_fold%d.dat.%s' % (data_name, data_name, i, mode))
#         new_labels = preparation.map_index_of_label('%s/hierarchy.pickle' % data_name, labels)
#         if mode == 'train':
#             train_data, validate_data, train_target, validate_target = train_test_split(
#                 datas, new_labels, test_size=0.1, random_state=12345)
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), train_data, train_target)
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, "validate"), validate_data, validate_target)
#         else:
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), datas, new_labels)

# Real Flow 

In [7]:
from data import Dataset
from embedding import Doc2Vec, NoTag_Doc2Vec
from assemble_classifier import AssemblePredicted, AssemblePredictedHidden, AssemblePredictedHiddenAdd
import shutil
import os
%load_ext autoreload
%autoreload 2

In [8]:
fold = 1

In [9]:
if os.path.isdir('data/%s/output' % data_name):
    shutil.rmtree('data/%s/output' % data_name)

In [ ]:
dataset_train = Dataset(data_name, fold, "train")
dataset_validate = Dataset(data_name, fold, "validate")
dataset_test = Dataset(data_name, fold, "test")

In [ ]:
%%time
doc2vec = Doc2Vec(dataset_train.number_of_classes(), size=200, epoch=3000, batch_size=10000)
doc2vec.fit(dataset_train.datas, dataset_train.labels, dataset_validate.datas, dataset_validate.labels, early_stopping=False)
# doc2vec.load_model("best_now/doc2vec.model")

Doc2Vec by Gensim
Epoch: 60 Similar: 355.74
Epoch: 120 Similar: 355.91
Epoch: 180 Similar: 416.02
Epoch: 240 Similar: 464.68
Epoch: 300 Similar: 498.79
Epoch: 360 Similar: 522.24
Epoch: 420 Similar: 543.14
Epoch: 480 Similar: 559.15
Epoch: 540 Similar: 574.16
Epoch: 600 Similar: 584.16
Epoch: 660 Similar: 595.19
Epoch: 720 Similar: 606.07
Epoch: 780 Similar: 614.86
Epoch: 840 Similar: 621.75
Epoch: 900 Similar: 630.84
Epoch: 960 Similar: 638.38
Epoch: 1020 Similar: 644.25
Epoch: 1080 Similar: 651.75
Epoch: 1140 Similar: 659.33
Epoch: 1200 Similar: 664.33
Epoch: 1260 Similar: 672.81
Epoch: 1320 Similar: 676.99
Epoch: 1380 Similar: 685.23
Epoch: 1440 Similar: 688.55
Epoch: 1500 Similar: 692.17
Epoch: 1560 Similar: 700.24
Epoch: 1620 Similar: 705.42
Epoch: 1680 Similar: 708.33
Epoch: 1740 Similar: 710.44
Epoch: 1800 Similar: 716.01
Epoch: 1860 Similar: 721.40
Epoch: 1920 Similar: 724.22
Epoch: 1980 Similar: 728.15
Epoch: 2040 Similar: 732.88
Epoch: 2100 Similar: 736.73
Epoch: 2160 Similar

In [ ]:
dataset_train.change_to_Doc2Vec(doc2vec)
dataset_validate.change_to_Doc2Vec(doc2vec)
dataset_test.change_to_Doc2Vec(doc2vec)

In [ ]:
model =  AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=2000, stopping_time=200, batch_size=65536, hidden_size=[300,1500,2000,3000,3000,2000,2000,1500,300], target_hidden_size=[60,100,150,200,300,300,200,100,60], use_dropout=True, start_level=6)
# model = AssembleNoLabel(data_name, dataset_train, dataset_validate, dataset_test, iteration=2000, batch_size=100, hidden_size=[300,1500,1500,1500], use_dropout=False, start_level=0, end_level=100)
# model = AssemblePredictedHidden(data_name, dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=2000, hidden_size=[300,1500,2000,2500], use_dropout=True, start_level=0, end_level=100)

In [ ]:
%%time
model.train()

In [ ]:
model.tuning_threshold()

In [ ]:
f1_macro, f1_micro, f1_each = model.evaluate("train", correction=False)
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))

In [ ]:
f1_macro, f1_micro, f1_each = model.evaluate("validate", correction=False)
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))

In [ ]:
f1_macro, f1_micro, f1_each = model.evaluate("test", correction=True)
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))